In [18]:
import pandas as pd

############ load and digitize the data
data=pd.read_csv('/home/ebotian/MCM/tennis.csv')
data = pd.get_dummies(data, columns=['winner_shot_type','serve_width','serve_depth','return_depth'])
############

############# pre-process the data
# fill nan with 0, and replace AD with 50
data = data.fillna(0)
data = data.replace('AD', 50.0)
data['point_victor']=data["point_victor"].replace(2,0)
#print(data.iloc[:,15])

# split the data into different match
grouped = dict(tuple(data.groupby(data['match_id'].ne(data['match_id'].shift()).cumsum())))

# Rename the subdata
subdata = {df['match_id'].iloc[0]: df for _, df in grouped.items()}

# Create a new dataset from the first column, excluding duplicates
match = pd.DataFrame(data.iloc[:, 0].drop_duplicates()).iloc[:,0].tolist()
#print(match_id[0])
##############


In [19]:
# the id of the match
id=0

In [20]:
# Calculate the time difference between consecutive rows

# Convert the timestamp column to datetime format
subdata[match[id]]['elapsed_time'] = pd.to_timedelta(subdata[match[id]]['elapsed_time'])

# Calculate the time difference between consecutive rows
subdata[match[id]]['time_diff'] = subdata[match[id]]['elapsed_time'].diff()

# Calculate the 5th and 95th percentiles
lower_threshold = subdata[match[id]]['time_diff'].quantile(0.05)
upper_threshold = subdata[match[id]]['time_diff'].quantile(0.95)

# Exclude the top 5% and bottom 5% of periods
filtered_diff = subdata[match[id]]['time_diff'][(subdata[match[id]]['time_diff'] > lower_threshold) & (subdata[match[id]]['time_diff'] < upper_threshold)]

# Calculate the average of the remaining intervals
average_interval = filtered_diff.mean()

print(f'Average interval (excluding top 5% and bottom 5%): {average_interval}')

# Convert the time differences to integer seconds
subdata[match[id]]['time_diff'] = subdata[match[id]]['time_diff'].dt.total_seconds()

# Replace 'NaT' values with 0
subdata[match[id]]['time_diff'] = subdata[match[id]]['time_diff'].fillna(0).astype(int)
#fill nan with 0
#subdata[match[id]]['time_diff'] = subdata[match[id]]['time_diff'].fillna(0)

Average interval (excluding top 5% and bottom 5%): 0 days 00:00:43.784090909


In [21]:
############ add features
add_feature=["score_diff"]

########## defining the new features

#subdata[match_id[0]][add_feature[0]] = subdata[match_id[0]]['p1_games'] - subdata[match_id[0]]['p2_games']

In [22]:
# split the data into features and target
target=pd.DataFrame(subdata[match[id]]["point_victor"])

subdata[match[id]]=subdata[match[id]].drop(columns=["point_victor"])
features=subdata[match[id]].iloc[:,4:]
#print(features)

In [25]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

# Assuming 'features' is your input data and 'target' is your output data...

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, shuffle=False)

# Reshape input data for LSTM (samples, timesteps, features)
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1])).astype(float)
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1])).astype(float)

# Convert target to numpy array
y_train = y_train.values
y_test = y_test.values

# Define LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(X_train, y_train, epochs=50, verbose=0)

# Make predictions for the test set
predictions = model.predict_classes(X_test)

AttributeError: 'Sequential' object has no attribute 'predict_classes'